In [11]:
# License: BSD
# Author: Sasank Chilamkurthy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import urllib.request
import zipfile

cudnn.benchmark = True
plt.ion()   # interactive mode

In [13]:
# Define the directory to download and extract the dataset
data_dir = 'hymenoptera_data'
zip_path = os.path.join(data_dir, 'hymenoptera_data.zip')

# Check if the dataset has already been downloaded and extracted
if not os.path.exists(data_dir):
    # If the directory does not exist, create it
    os.makedirs(data_dir, exist_ok=True)

    # Download the dataset zip file
    url = 'https://download.pytorch.org/tutorial/hymenoptera_data.zip'
    print("Downloading the dataset...")
    urllib.request.urlretrieve(url, zip_path)

    # Unzip the dataset into the specified directory
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

    # Remove the zip file after extraction to save space
    os.remove(zip_path)

    print(f"Dataset downloaded and extracted to '{data_dir}'")
else:
    print(f"Dataset already exists in '{data_dir}'. Skipping download.")


Dataset already exists in 'hymenoptera_data'. Skipping download.


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




In [21]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [23]:
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\mihir/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:21<00:00, 2.21MB/s]


In [24]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.4822 Acc: 0.7418
val Loss: 0.1985 Acc: 0.9281

Epoch 1/24
----------
train Loss: 0.5296 Acc: 0.7910
val Loss: 0.3357 Acc: 0.8693

Epoch 2/24
----------
train Loss: 0.5177 Acc: 0.7869
val Loss: 0.3815 Acc: 0.8170

Epoch 3/24
----------
train Loss: 0.5188 Acc: 0.7992
val Loss: 0.4918 Acc: 0.8301

Epoch 4/24
----------
train Loss: 0.4603 Acc: 0.7787
val Loss: 0.3100 Acc: 0.8889

Epoch 5/24
----------
train Loss: 0.6975 Acc: 0.7418
val Loss: 0.8011 Acc: 0.7451

Epoch 6/24
----------
train Loss: 0.5644 Acc: 0.7910
val Loss: 0.2775 Acc: 0.9085

Epoch 7/24
----------
train Loss: 0.3742 Acc: 0.8648
val Loss: 0.2538 Acc: 0.9150

Epoch 8/24
----------
train Loss: 0.3245 Acc: 0.8811
val Loss: 0.2897 Acc: 0.8954

Epoch 9/24
----------
train Loss: 0.3732 Acc: 0.8525
val Loss: 0.2737 Acc: 0.9216

Epoch 10/24
----------
train Loss: 0.2749 Acc: 0.8934
val Loss: 0.2168 Acc: 0.9346

Epoch 11/24
----------
train Loss: 0.3057 Acc: 0.8525
val Loss: 0.2149 Acc: 0.9281

Ep

In [27]:
model_conv = torchvision.models.resnet18(weights='IMAGENET1K_V1')
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5584 Acc: 0.6680
val Loss: 0.3281 Acc: 0.8497

Epoch 1/24
----------
train Loss: 0.5618 Acc: 0.7459
val Loss: 0.1719 Acc: 0.9346

Epoch 2/24
----------
train Loss: 0.3883 Acc: 0.8197
val Loss: 0.2604 Acc: 0.9085

Epoch 3/24
----------
train Loss: 0.5427 Acc: 0.7910
val Loss: 0.5863 Acc: 0.7843

Epoch 4/24
----------
train Loss: 0.4777 Acc: 0.8156
val Loss: 0.1923 Acc: 0.9412

Epoch 5/24
----------
train Loss: 0.5313 Acc: 0.7869
val Loss: 0.1573 Acc: 0.9542

Epoch 6/24
----------
train Loss: 0.3987 Acc: 0.8115
val Loss: 0.1677 Acc: 0.9412

Epoch 7/24
----------
train Loss: 0.3141 Acc: 0.8730
val Loss: 0.1641 Acc: 0.9477

Epoch 8/24
----------
train Loss: 0.4130 Acc: 0.8484
val Loss: 0.1712 Acc: 0.9412

Epoch 9/24
----------
train Loss: 0.3610 Acc: 0.8279
val Loss: 0.1736 Acc: 0.9412

Epoch 10/24
----------
train Loss: 0.3828 Acc: 0.8361
val Loss: 0.1676 Acc: 0.9542

Epoch 11/24
----------
train Loss: 0.3466 Acc: 0.8484
val Loss: 0.1514 Acc: 0.9608

Ep

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    
    # Turn off gradients for validation, saves memory and computations
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)  # Send inputs to the device (GPU or CPU)
            labels = labels.to(device)  # Send labels to the device
            
            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class with max probability
            
            total += labels.size(0)  # Update the total number of samples
            correct += (predicted == labels).sum().item()  # Count correct predictions
            
    accuracy = 100 * correct / total  # Calculate accuracy percentage
    return accuracy

# Report accuracy for Fine-Tuned Model
train_acc_ft = evaluate_model(model_ft, dataloaders['train'])
val_acc_ft = evaluate_model(model_ft, dataloaders['val'])
print(f'Fine-Tuned Model Training Accuracy: {train_acc_ft:.2f}%')
print(f'Fine-Tuned Model Validation Accuracy: {val_acc_ft:.2f}%')

# Report accuracy for Feature Extractor Model
train_acc_fe = evaluate_model(model_conv, dataloaders['train'])
val_acc_fe = evaluate_model(model_conv, dataloaders['val'])
print(f'Feature Extractor Model Training Accuracy: {train_acc_fe:.2f}%')
print(f'Feature Extractor Model Validation Accuracy: {val_acc_fe:.2f}%')


Fine-Tuned Model Training Accuracy: 98.77%
Fine-Tuned Model Validation Accuracy: 93.46%
Feature Extractor Model Training Accuracy: 92.62%
Feature Extractor Model Validation Accuracy: 96.08%
